In [21]:
print("Start")

import pandas as pd

data = pd.read_csv("../../../dataset-sis-energy-derived-reanalysis-temperature-country_daily/temp_country_daily.csv", sep=";")

Start


In [22]:
# data.head()
# data.info()

In [23]:
df = data[["Date", "DE"]].copy()
df

,Date,DE
0,1979-01-01,260.148301
1,1979-01-02,261.597371
2,1979-01-03,264.019374
3,1979-01-04,262.404894
4,1979-01-05,260.625696
...,...,...
16309,2023-08-27,288.666687
16310,2023-08-28,287.271409
16311,2023-08-29,286.991303
16312,2023-08-30,287.637229


In [24]:
# convert column holding the date from Object to Datetime
df["Date"] = pd.to_datetime(df["Date"])

# resample from daily to week data and aggregate on mean temperature values over the week
# not resample("7D") as before as overflow to following year happens
# df = df.resample("1W", on="Date").max()

# convert temperature values from Kelvon to Celsius
df.DE = df.DE - 273.15

In [25]:
# reset index from date value to integer 
df = df.reset_index()
# get weeknumber as separate column
df["weeknumber"] = df.Date.dt.isocalendar().week
# get year number as separate column
df["year"] = df.Date.dt.isocalendar().year

In [26]:
# view data
# df

In [27]:
# set to "hot", when temp higher than 20 degrees
compareToInCelsius = 20
df["DE_masked"] = df.DE.mask(df.DE > compareToInCelsius, "hot")

In [28]:
grouped = df.groupby(["year", "weeknumber"])
grouped.describe()

index                                                 \
                count     mean       std      min       25%      50%   
year weeknumber                                                        
1979 1            7.0      3.0  2.160247      0.0      1.50      3.0   
     2            7.0     10.0  2.160247      7.0      8.50     10.0   
     3            7.0     17.0  2.160247     14.0     15.50     17.0   
     4            7.0     24.0  2.160247     21.0     22.50     24.0   
     5            7.0     31.0  2.160247     28.0     29.50     31.0   
...               ...      ...       ...      ...       ...      ...   
2023 31           7.0  16285.0  2.160247  16282.0  16283.50  16285.0   
     32           7.0  16292.0  2.160247  16289.0  16290.50  16292.0   
     33           7.0  16299.0  2.160247  16296.0  16297.50  16299.0   
     34           7.0  16306.0  2.160247  16303.0  16304.50  16306.0   
     35           4.0  16311.5  1.290994  16310.0  16310.75  16311.5   

                                      DE                                  \
                      75%      max count       mean       std        min   
year weeknumber                                                            
1979 1               4.50      6.0   7.0 -11.083423  1.852741 -13.001699   
     2              11.50     13.0   7.0  -1.082420  0.903849  -2.306801   
     3              18.50     20.0   7.0  -5.514743  0.499501  -6.290636   
     4              25.50     27.0   7.0  -2.497242  1.597949  -5.778707   
     5              32.50     34.0   7.0   0.024851  1.014254  -1.508023   
...                   ...      ...   ...        ...       ...        ...   
2023 31          16286.50  16288.0   7.0  16.523716  0.926717  14.624359   
     32          16293.50  16295.0   7.0  17.195079  2.882499  13.864635   
     33          16300.50  16302.0   7.0  21.855895  1.318487  19.709580   
     34          16307.50  16309.0   7.0  20.099997  2.392609  15.516687   
     35          16312.25  16313.0   4.0  14.062419  0.317195  13.799736   

                                                             
                       25%        50%        75%        max  
year weeknumber                                              
1979 1          -12.509871 -11.552629  -9.937866  -8.134158  
     2           -1.843465  -0.642088  -0.462624  -0.015873  
     3           -5.742158  -5.484950  -5.275706  -4.791886  
     4           -2.769466  -2.102620  -1.583980  -0.892475  
     5           -0.291591  -0.214165   0.336384   1.806560  
...                    ...        ...        ...        ...  
2023 31          16.379582  16.782519  17.046273  17.407422  
     32          14.870508  16.226245  20.018513  20.496633  
     33          21.292183  21.929622  22.550667  23.666362  
     34          19.525953  20.789899  21.496327  22.348831  
     35          13.830911  13.981356  14.212864  14.487229  

[2331 rows x 16 columns]

In [29]:
df_grouped = grouped["DE"].max().to_frame(name="max_daily_in_week").reset_index()
df_grouped

,year,weeknumber,max_daily_in_week
0,1979,1,-8.134158
1,1979,2,-0.015873
2,1979,3,-4.791886
3,1979,4,-0.892475
4,1979,5,1.806560
...,...,...,...
2326,2023,31,17.407422
2327,2023,32,20.496633
2328,2023,33,23.666362
2329,2023,34,22.348831


In [30]:
# save result to a csv file
df_grouped.to_json('temp_country_weekly_max_daily_in_week.json', orient='records')
